In [1]:
import time

import torch
import tool.tool as tool
from dataTreat.readData import LzgdDataset,img_transforms
import numpy as np
from torch.utils.data import Dataset,DataLoader
from torchvision import transforms
import os,cv2
import numpy as np
import tool.visualizationTool as visTool


train_img_dir='../data/trainImgPart'
train_label_dir='../data/trainLabel1Part'

trainDataset = LzgdDataset(train_img_dir,train_label_dir,img_transforms)
trainDataloader = DataLoader(trainDataset, batch_size=1, shuffle=True, num_workers=0, pin_memory=True)
type(trainDataset),len(trainDataset)

valid_img_dir='../data/validImg'
valid_label_dir='../data/validLabel1'

validDataset = LzgdDataset(valid_img_dir,valid_label_dir,img_transforms)
validDataloader = DataLoader(validDataset, batch_size=1, shuffle=True, num_workers=0, pin_memory=True)
print('读取数据完成')
#
img,label=trainDataloader.dataset[0]
print(label.shape,label.dtype)
# tool.getValueCount(label)

读取数据完成
torch.Size([3, 256, 256]) torch.float32


In [2]:
import torch
from torch import nn,optim
import pandas as pd
import numpy as np
from collections import Counter
import tool.tool as tool
from module.module import Module
from module.lzgdModule import Lzgd
from module.FCNModule import FCN
from module.unet import UNet
from indicator import evaluate as eva
from module.unetplusplusModule import NestedUNet
from loss.dice import BCE_DICE_Loss
import cv2
import segmentation_models_pytorch as smp


torch.cuda.is_available()
torch.manual_seed(42)
img,label=trainDataloader.dataset[0]
# visTool.showImage(label)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#超参数
lr=1e-3
criterion = nn.BCEWithLogitsLoss().cuda()

model_unet=Module(model=UNet(n_channels=3,n_classes=3).cuda(),criterion=criterion,LR=lr,flag='unet')
model=model_unet.model
# model=smp.Unet(encoder_name='resnet50',in_channels=3,classes=3).to(device)


optimizer = torch.optim.Adam(params=model.parameters(),lr=lr) #1e-3
# optimizer = torch.optim.SGD(params=model.parameters(),lr=lr) #lr=1e-2





epochs = 500
tj=None
pred_list=[]
np_list=[]

for epoch in range(epochs):
    img=tool.Dim3to4(img)
    label=tool.Dim3to4(label)
    model.train()
    img=img.to(device=device)
    pred=model(img)
    # print(pred.shape,pred.dtype,label.shape,label.dtype)
    # pred=pred.sigmoid()
    pred_list.append(pred)
    # tool.getValueCount(label)


    loss = criterion(pred, label.to(device=device))

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if epoch % 20==0 or epoch==epochs-1:
        print(epoch,loss.data)


    aa=tool.tensor2numpy(pred)*255
    np_list.append(aa)
    cv2.imwrite(f'../data/data_pred1/{str(epoch)}.png',aa)

D:\paper\code
Accuracy: 200.00%
Recall: 100.00%
Precision: 100.00%
0 tensor(0.7473, device='cuda:0')
20 tensor(0.3696, device='cuda:0')
40 tensor(0.2499, device='cuda:0')
60 tensor(0.1653, device='cuda:0')
80 tensor(0.1115, device='cuda:0')
100 tensor(0.0785, device='cuda:0')
120 tensor(0.0578, device='cuda:0')
140 tensor(0.0441, device='cuda:0')
160 tensor(0.0348, device='cuda:0')
180 tensor(0.0280, device='cuda:0')
200 tensor(0.0232, device='cuda:0')
220 tensor(0.0194, device='cuda:0')
240 tensor(0.0165, device='cuda:0')
260 tensor(0.0144, device='cuda:0')
280 tensor(0.0124, device='cuda:0')
300 tensor(0.0108, device='cuda:0')
320 tensor(0.0097, device='cuda:0')
340 tensor(0.0086, device='cuda:0')
360 tensor(0.0077, device='cuda:0')
380 tensor(0.0069, device='cuda:0')
400 tensor(0.0070, device='cuda:0')
420 tensor(0.0061, device='cuda:0')
440 tensor(0.0055, device='cuda:0')
460 tensor(0.0050, device='cuda:0')
480 tensor(0.0046, device='cuda:0')
499 tensor(0.0043, device='cuda:0')


In [15]:
len(model.state_dict())

278

In [11]:
path='../data/data_pred'
os.path.exists(path)

img_check=pred_list[499]
# img_check=tool.fitPredLabel(img_check)
print(f"maxValues:{img_check.max().data}",f"minValues:{img_check.min().data}")
visTool.showImage(img_check)
#
# tool.getValueCount(img_check)
# ouputImg=tool.tensor2numpy(pred_list[190])*255
# ouputImg.shape
# tool.getValueCount(ouputImg)
# cv2.imwrite(r'190.png',ouputImg)

IndexError: list index out of range

In [10]:
file='../data/trainLabel1Part/randomColor04401172210002DDOM.png'
img=cv2.imread(file)
img=torch.from_numpy(img)
print(type(img))
# tool.getValueCount(img)

<class 'torch.Tensor'>
